In [5]:
import numpy as np
import cvxpy as cp
import pandas as pd
import mosek
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import scipy.stats
from scipy.optimize import minimize
from scipy.stats import weibull_min
import cyipopt

In [6]:
def kl_c(x):
    return(np.exp(x)-1)

def d_kl_c(x):
    return(np.exp(x))

def dd_kl_c(x):
    return(np.exp(x))

def kl_d1dl(t1,s_vec,lbda,a,b,N):
    som = 0
    for i in range(N):
        fac1 = -(s_vec[i]+t1)/(lbda**2)
        fac2 = 1/N*dd_kl_c((s_vec[i]+t1)/lbda)
        som = som + fac1*fac2 
    return(som)

def kl_d1ds(t1,s,lbda,a,b,N):
    return(1/N*dd_kl_c((s+t1)/lbda)*1/lbda)

def kl_dsdl(t1,s,lbda,a,b,N):
    return(1/N*(-s-t1)/(lbda**2)*dd_kl_c((s+t1)/lbda))

def kl_dll(t1,s_vec,lbda,a,b,N):
    som = 0
    for i in range(N):
        som = som + (s_vec[i]+t1)**2/(lbda**3)*dd_kl_c((s_vec[i]+t1)/lbda)
    return(som/N)

def kl_d11(t1,s_vec,lbda,a,b,N):
    som = 0
    for i in range(N):
        som = som + dd_kl_c((s_vec[i]+t1)/lbda)/lbda
    return(som/N)
def kl_dss(t1,s,lbda,a,b,N):
    return(1/N*dd_kl_c((s+t1)/lbda)*1/lbda)


def log_bar(x):
    if x>0:
        return(-np.log(x))
    else:
        return(np.inf)

In [27]:
np.random.seed(3)
X = -(np.random.lognormal(0,1,size= 5))
a = 0.01
b = 2
alpha = 0.5
r = 0.1
para = [a,b,alpha,r]
f = np.zeros(len(X)) + 1/len(X)
mu = 0.001

In [28]:
###### Robust KL-CVaR_barrier

class HS074_b(cyipopt.Problem):
    
    
    def objective(self, x):
        """Returns the scalar value of the objective given x."""
        #a,b,alpha,r = self.para
        the1 = x[0]
        the2 = x[1]
        lbda = x[2]
        N = len(X)
        som = 0
        sb_1 = 0
        for i in range(3,N+3):
            som = som + lbda * kl_c((x[i]+the1)/lbda)
            sb_1 = sb_1 +log_bar(x[i]-1/alpha*(x[1]-X[i-3])) + log_bar(x[i])
        return(-the1-the2+lbda*r+som/N +mu*sb_1+mu*log_bar(x[2]))

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        #a,b,alpha,r = self.para
        the1 = x[0]
        the2 = x[1]
        lbda = x[2]
        N = len(X)
        s1 = 0
        s2 = 0
        sb1 = 0
        grad = np.zeros(N+3) 
        for i in range(3,N+3):
            grad[i] = d_kl_c((x[i]+the1)/lbda)/N-mu/(x[i]-1/alpha*(x[1]-X[i-3]))-mu/x[i]
            s1 = s1 + d_kl_c((x[i]+the1)/lbda)/N
            s2 = s2 + kl_c((x[i]+the1)/lbda) - d_kl_c((x[i]+the1)/lbda)*((x[i]+the1)/lbda)
            sb1 = sb1 + mu/(alpha*x[i]-x[1]+X[i-3])
        grad[1] = -1 + sb1
        grad[0] = -1 + s1
        grad[2] = r + s2/N - mu/lbda
        
        return(grad)
        
       
    def constraints(self, x):
        """Returns the constraints."""
        cons = np.array([x[2]])
        return(cons)

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        
        
        jacob = np.zeros(N+3)
        jacob[2] = 1
        jacob = np.array([jacob])
        
        row, col = self.jacobianstructure()
        return(jacob[row,col])
        
        
    def jacobianstructure(self):
        N = len(X)
        jac_struc = np.zeros(N+3)
        jac_struc[2] = 1
        jac_struc = np.array([jac_struc])
        return(np.nonzero(jac_struc))
        
        
    
    
    def hessianstructure(self):
        """Returns the row and column indices for non-zero vales of the
        Hessian."""

        # NOTE: The default hessian structure is of a lower triangular matrix,
        # therefore this function is redundant. It is included as an example
        # for structure callback.
        #a,b,alpha,r = self.para
        N = len(X)
        struc = np.tril(np.ones((N+3,N+3)))
        struc[1,:] = 0
        struc[0:N+3,1] = 0
        for i in range(3,N+1):
            struc[(i+1):(N+3),i] = 0
        struc[N+2,N+1] = 0
        return np.nonzero(struc)

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""
        #a,b,alpha,r = self.para
        t1 = x[0]
        t2 = x[1]
        lbda = x[2]
        N = len(X)
        s1 = 0
        s2 = 0
        s3 = 0
        s_vec = x[3:(N+3)]
        a = 1
        b = 1
        H = np.zeros((N+3,N+3))
        H[0,0] = kl_d11(t1,s_vec,lbda,a,b,N)
        H[2,2] = kl_dll(t1,s_vec,lbda,a,b,N) + mu/lbda**2
        H[2,0] = kl_d1dl(t1,s_vec,lbda,a,b,N)
        for i in range(3,N+3):
            s2 = s2 + 1/(alpha**2*(x[i]-1/alpha*(t2-X[i-3]))**2)
            H[i,0] = kl_d1ds(t1,x[i],lbda,a,b,N)
            H[i,2] = kl_dsdl(t1,x[i],lbda,a,b,N)
            H[i,i] = kl_dss(t1,x[i],lbda,a,b,N)+mu/((x[i]-1/alpha*(t2-X[i-3]))**2)+mu/x[i]**2

        H[1,1] = s2*mu

        H = obj_factor*H

        row, col = self.hessianstructure()

        return H[row, col]


    def intermediate(self, alg_mod, iter_count, obj_value, inf_pr, inf_du, mu,
                     d_norm, regularization_size, alpha_du, alpha_pr,
                     ls_trials):
        """Prints information at every Ipopt iteration."""
        iterate = self.get_current_iterate()
        infeas = self.get_current_violations()
        primal = iterate["x"]
        jac = self.jacobian(primal)
        msg = "Objective value at iteration #{:d} is - {:g}"

        print(msg.format(iter_count, obj_value))
        print("Primal iterate:", primal)
        #infeasi = []
        #for i in range(len(X)):
        #    infeasi.append(1/alpha * (primal[1]-X[i])-primal[3+i])
       # print('infeasi:', np.max(infeasi))
        print('inf_pr:', inf_pr)
        #print("Flattened Jacobian:", jac)
        #print("Dual infeasibility:", infeas["grad_lag_x"])

In [23]:
lb = [-10,-10,0.01]
ub = [10, 10, 10]


cl = [0.0001]
cu = [500]
x0 = [-2,np.min(X),1]
N = len(X)
for i in range(3, N+3):
    lb.append(0)
    ub.append(1000)
    x0.append(np.maximum(1/alpha*(x0[1]-X[i-3]),0)+0.01)

In [29]:
x0 = x

In [30]:
nlp = HS074_b(
    n=len(x0),
    m=len(cl),
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu,
)
nlp.add_option('tol', 1e-6)
nlp.add_option('max_iter', 6000)

In [31]:
x, info = nlp.solve(x)

Objective value at iteration #0 is - 6.17212
Primal iterate: [-0.17300934 -5.92917426  0.83151496  0.12433671  0.16313418  0.16308646
  0.16300905  0.16305517]
inf_pr: 0.0
Objective value at iteration #1 is - 6.16728
Primal iterate: [-0.1730922  -5.92335797  0.83161666  0.12446021  0.16326571  0.16321801
  0.16314064  0.16318673]
inf_pr: 0.0
Objective value at iteration #2 is - 6.16562
Primal iterate: [-0.17310732 -5.91950931  0.83163692  0.12446745  0.16328745  0.16323975
  0.16316238  0.16320847]
inf_pr: 0.0
Objective value at iteration #3 is - 6.16552
Primal iterate: [-0.17311092 -5.92049556  0.8316424   0.12446752  0.1632928   0.16324511
  0.16316774  0.16321383]
inf_pr: 0.0
Objective value at iteration #4 is - 6.16543
Primal iterate: [-0.1731114  -5.92001456  0.83164321  0.12446755  0.16329348  0.16324578
  0.16316841  0.1632145 ]
inf_pr: 0.0
Objective value at iteration #5 is - 6.16543
Primal iterate: [-0.17311149 -5.9199845   0.83164335  0.12446755  0.1632936   0.1632459
  0.163

Objective value at iteration #95 is - 6.16601
Primal iterate: [-0.17354578 -5.91997595  0.83318352  0.12447727  0.16384731  0.16379968
  0.16372242  0.16376845]
inf_pr: 0.0
Objective value at iteration #96 is - 6.16601
Primal iterate: [-0.17354577 -5.92000677  0.83318347  0.12447727  0.1638473   0.16379967
  0.16372242  0.16376844]
inf_pr: 0.0
Objective value at iteration #97 is - 6.16601
Primal iterate: [-0.17354574 -5.91999925  0.8331833   0.12447728  0.16384728  0.16379965
  0.16372239  0.16376842]
inf_pr: 0.0
Objective value at iteration #98 is - 6.16601
Primal iterate: [-0.17354572 -5.92000695  0.83318318  0.12447728  0.16384726  0.16379963
  0.16372238  0.1637684 ]
inf_pr: 0.0
Objective value at iteration #99 is - 6.16601
Primal iterate: [-0.17354558 -5.91999386  0.83318236  0.12447728  0.16384715  0.16379952
  0.16372226  0.16376829]
inf_pr: 0.0
Objective value at iteration #100 is - 6.16601
Primal iterate: [-0.17354556 -5.92000927  0.83318229  0.12447728  0.16384714  0.16379951

Objective value at iteration #183 is - 6.16598
Primal iterate: [-0.17351891 -5.92000012  0.83302766  0.12447795  0.1638258   0.16377819
  0.16370096  0.16374696]
inf_pr: 0.0
Objective value at iteration #184 is - 6.16598
Primal iterate: [-0.17351665 -5.92003832  0.83301452  0.12447801  0.16382399  0.16377637
  0.16369915  0.16374515]
inf_pr: 0.0
Objective value at iteration #185 is - 6.16598
Primal iterate: [-0.17351645 -5.9199782   0.83301335  0.12447801  0.16382382  0.16377621
  0.16369898  0.16374499]
inf_pr: 0.0
Objective value at iteration #186 is - 6.16598
Primal iterate: [-0.17351644 -5.92000902  0.83301329  0.12447801  0.16382382  0.1637762
  0.16369898  0.16374498]
inf_pr: 0.0
Objective value at iteration #187 is - 6.16598
Primal iterate: [-0.17351641 -5.91999399  0.83301315  0.12447801  0.1638238   0.16377619
  0.16369896  0.16374497]
inf_pr: 0.0
Objective value at iteration #188 is - 6.16598
Primal iterate: [-0.17351641 -5.92000169  0.83301314  0.12447801  0.1638238   0.1637

Objective value at iteration #271 is - 6.16594
Primal iterate: [-0.1734866  -5.91998355  0.83284013  0.12447876  0.16379992  0.16375233
  0.16367513  0.16372112]
inf_pr: 0.0
Objective value at iteration #272 is - 6.16594
Primal iterate: [-0.17348649 -5.92001436  0.83283953  0.12447877  0.16379984  0.16375224
  0.16367505  0.16372104]
inf_pr: 0.0
Objective value at iteration #273 is - 6.16594
Primal iterate: [-0.17348593 -5.9199888   0.83283625  0.12447878  0.16379938  0.16375179
  0.16367459  0.16372058]
inf_pr: 0.0
Objective value at iteration #274 is - 6.16594
Primal iterate: [-0.17348592 -5.9200042   0.8328362   0.12447878  0.16379938  0.16375178
  0.16367459  0.16372058]
inf_pr: 0.0
Objective value at iteration #275 is - 6.16594
Primal iterate: [-0.17348585 -5.91999568  0.83283579  0.12447878  0.16379932  0.16375173
  0.16367453  0.16372052]
inf_pr: 0.0
Objective value at iteration #276 is - 6.16594
Primal iterate: [-0.17348574 -5.92001109  0.83283515  0.12447878  0.16379923  0.163

Objective value at iteration #357 is - 6.16586
Primal iterate: [-0.17341578 -5.91998782  0.83242912  0.12448054  0.1637432   0.16369565
  0.16361852  0.16366447]
inf_pr: 0.0
Objective value at iteration #358 is - 6.16586
Primal iterate: [-0.1734156  -5.92001864  0.83242806  0.12448055  0.16374305  0.1636955
  0.16361838  0.16366432]
inf_pr: 0.0
Objective value at iteration #359 is - 6.16586
Primal iterate: [-0.17341503 -5.91998481  0.83242478  0.12448056  0.1637426   0.16369505
  0.16361793  0.16366387]
inf_pr: 0.0
Objective value at iteration #360 is - 6.16586
Primal iterate: [-0.17341502 -5.92001563  0.8324247   0.12448056  0.16374259  0.16369504
  0.16361791  0.16366386]
inf_pr: 0.0
Objective value at iteration #361 is - 6.16586
Primal iterate: [-0.17341497 -5.91998557  0.83242443  0.12448056  0.16374255  0.163695
  0.16361788  0.16366382]
inf_pr: 0.0
Objective value at iteration #362 is - 6.16586
Primal iterate: [-0.17341497 -5.92000097  0.83242442  0.12448056  0.16374255  0.163695

...

Objective value at iteration #5684 is - 6.13665
Primal iterate: [-0.14863866 -5.92150186  0.68684984  0.1214792   0.14361187  0.14358001
  0.14352831  0.14355911]
inf_pr: 0.0
Objective value at iteration #5685 is - 6.13665
Primal iterate: [-0.14863799 -5.92148189  0.68684553  0.12147923  0.14361128  0.14357942
  0.14352772  0.14355852]
inf_pr: 0.0
Objective value at iteration #5686 is - 6.13665
Primal iterate: [-0.14863791 -5.92151269  0.68684505  0.12147923  0.14361122  0.14357935
  0.14352766  0.14355846]
inf_pr: 0.0
Objective value at iteration #5687 is - 6.13665
Primal iterate: [-0.14863724 -5.92147668  0.68684074  0.12147926  0.14361063  0.14357876
  0.14352707  0.14355787]
inf_pr: 0.0
Objective value at iteration #5688 is - 6.13665
Primal iterate: [-0.14863724 -5.92150749  0.6868407   0.12147926  0.14361062  0.14357876
  0.14352706  0.14355786]
inf_pr: 0.0
Objective value at iteration #5689 is - 6.13665
Primal iterate: [-0.1486372  -5.92149246  0.68684049  0.12147926  0.14361059 

Objective value at iteration #5774 is - 6.13662
Primal iterate: [-0.14861629 -5.92150736  0.6867064   0.12148015  0.14359227  0.14356042
  0.14350875  0.14353953]
inf_pr: 0.0
Objective value at iteration #5775 is - 6.13662
Primal iterate: [-0.14861562 -5.92147492  0.68670208  0.12148018  0.14359168  0.14355983
  0.14350816  0.14353894]
inf_pr: 0.0
Objective value at iteration #5776 is - 6.13662
Primal iterate: [-0.14861562 -5.92150573  0.68670205  0.12148018  0.14359167  0.14355982
  0.14350815  0.14353894]
inf_pr: 0.0
Objective value at iteration #5777 is - 6.13662
Primal iterate: [-0.1486156  -5.92149821  0.68670195  0.12148018  0.14359166  0.14355981
  0.14350814  0.14353892]
inf_pr: 0.0
Objective value at iteration #5778 is - 6.13662
Primal iterate: [-0.1486156  -5.92150206  0.68670192  0.12148018  0.14359166  0.14355981
  0.14350814  0.14353892]
inf_pr: 0.0
Objective value at iteration #5779 is - 6.13662
Primal iterate: [-0.14861543 -5.92149201  0.68670084  0.12148019  0.14359151 

Objective value at iteration #5856 is - 6.13657
Primal iterate: [-0.1485797  -5.92150208  0.68647168  0.1214817   0.14356019  0.14352836
  0.14347673  0.14350749]
inf_pr: 0.0
Objective value at iteration #5857 is - 6.13657
Primal iterate: [-0.14857936 -5.92148385  0.68646952  0.12148172  0.14355989  0.14352807
  0.14347644  0.1435072 ]
inf_pr: 0.0
Objective value at iteration #5858 is - 6.13657
Primal iterate: [-0.1485793  -5.92151465  0.68646915  0.12148172  0.14355984  0.14352802
  0.14347639  0.14350715]
inf_pr: 0.0
Objective value at iteration #5859 is - 6.13657
Primal iterate: [-0.14857897 -5.92148139  0.68646699  0.12148173  0.14355955  0.14352772
  0.14347609  0.14350685]
inf_pr: 0.0
Objective value at iteration #5860 is - 6.13657
Primal iterate: [-0.14857896 -5.9215122   0.68646695  0.12148174  0.14355954  0.14352772
  0.14347609  0.14350685]
inf_pr: 0.0
Objective value at iteration #5861 is - 6.13657
Primal iterate: [-0.14857895 -5.92149717  0.68646687  0.12148174  0.14355953 

Objective value at iteration #5949 is - 6.13655
Primal iterate: [-0.14856346 -5.92148238  0.68636752  0.12148239  0.14354595  0.14351414
  0.14346252  0.14349328]
inf_pr: 0.0
Objective value at iteration #5950 is - 6.13655
Primal iterate: [-0.14856345 -5.92151319  0.68636746  0.12148239  0.14354595  0.14351413
  0.14346252  0.14349327]
inf_pr: 0.0
Objective value at iteration #5951 is - 6.13655
Primal iterate: [-0.14856344 -5.92149816  0.68636739  0.1214824   0.14354594  0.14351412
  0.14346251  0.14349326]
inf_pr: 0.0
Objective value at iteration #5952 is - 6.13655
Primal iterate: [-0.14856344 -5.92150008  0.68636738  0.1214824   0.14354593  0.14351412
  0.14346251  0.14349326]
inf_pr: 0.0
Objective value at iteration #5953 is - 6.13655
Primal iterate: [-0.14855806 -5.92143096  0.6863329   0.12148262  0.14354122  0.14350941
  0.1434578   0.14348855]
inf_pr: 0.0
Objective value at iteration #5954 is - 6.13655
Primal iterate: [-0.14855798 -5.92155418  0.68633238  0.12148263  0.14354115 

Objective value at iteration #5998 is - 6.13652
Primal iterate: [-0.14853639 -5.92150364  0.68619389  0.12148355  0.14352222  0.14349043
  0.14343884  0.14346958]
inf_pr: 0.0
Objective value at iteration #5999 is - 6.13652
Primal iterate: [-0.14853637 -5.92149651  0.68619375  0.12148355  0.14352221  0.14349041
  0.14343882  0.14346956]
inf_pr: 0.0
Objective value at iteration #6000 is - 6.13652
Primal iterate: [-0.14853637 -5.92150036  0.68619371  0.12148355  0.1435222   0.1434904
  0.14343882  0.14346955]
inf_pr: 0.0


In [26]:
x

array([-0.17300934, -5.92917426,  0.83151496,  0.12433671,  0.16313418,
        0.16308646,  0.16300905,  0.16305517])

In [16]:
info

{'x': array([-10.        , -10.        ,   9.99999953,   0.13631112,
          0.13515425,   0.13510167,   0.13500589,   0.13506462]),
 'g': array([9.99999953]),
 'obj_val': 14.668551536163672,
 'mult_g': array([-8.90527961e-09]),
 'mult_x_L': array([0.00000000e+00, 0.00000000e+00, 9.18254101e-09, 6.69176401e-07,
        6.74904787e-07, 6.75167467e-07, 6.75646494e-07, 6.75352653e-07]),
 'mult_x_U': array([6.27108316e-01, 9.93181220e-01, 1.60262885e-01, 9.09214419e-11,
        9.09213370e-11, 9.09213323e-11, 9.09213236e-11, 9.09213289e-11]),
 'status': 0,
 'status_msg': b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'}